In [1]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from GA import *
from module import *
from IPython.display import clear_output
from tensorflow.keras.models import load_model

In [7]:
inveser_data = pd.read_csv("C:/Users/kunkun/Desktop/PythenData/train/32cases_2.csv" ,header=None)
inveser_data = np.array(inveser_data)

In [8]:
model = load_model('./train_ANN/my_model4.h5')

In [9]:
# Evolution strategy hyperparameters
#初始化参数
N = 200  # 种群内个体数目
N_chrom = 9  # 染色体节点数，也就是每个个体有多少条染色体，就是看适应函数里有几个自变量。
iter_all = 100  # 迭代次数，也就是一共有多少代
mut =0.6 # 突变概率
acr = 0.6  # 交叉概率
best = 1
chrom_range = np.array([[9.1, 11], [2.4, 5.5], [1.1, 3], [7.8, 10], [2.9, 4], [3.1, 5], [3.1, 5], [0.5, 3], [0.5, 3]])  # 每个节点的值的区间
chrom = np.ones((N, N_chrom))  # 存放染色体的矩阵
fitness = np.zeros((N, 1))  # 存放染色体的适应度
# fitness_ave = np.ones((1,iter_all))  # 存放每一代的平均适应度
fitness_best = np.ones((inveser_data.shape[0],iter_all))  # 存放每一代的最优适应度
chrom_best = np.ones((inveser_data.shape[0],N_chrom+1))  # 存放当前代的最优染色体与适应度
target_error_temp=100
best_pre_geo = pd.DataFrame(np.zeros((inveser_data.shape[0], N_chrom)))

In [10]:
for j in range(inveser_data.shape[0]):
    # 初始化，这只是用于生成第一代个体，并计算其适应度函数
    chrom = Initialize(N, N_chrom, chrom_range) #初始化染色体
    #创建第一代种群表格
    op_table = pd.DataFrame(chrom)
    
    #预测第一代种群的结果
    predicted_curves = model.predict(op_table.iloc[:,:])
    
    # estimate error based on true values
    target_scale = mean_squared_error(inveser_data[j, :], 0*inveser_data[j, :])
    target_error = [mean_squared_error(inveser_data[j, :], predicted_curves[k, :])/target_scale for k in range(0, predicted_curves.shape[0])]
    
    # add a column for error into the table
    op_table.insert(0, "error", target_error, True)
    # sort based on error
    op_table = op_table.sort_values(by="error")
    
    if target_error_temp > np.min(op_table.iloc[:, 0]):
        target_error_temp = np.min(op_table.iloc[:, 0])
        best_pre_geo.iloc[0, :] = op_table.iloc[0, 1:]
    
    #适应度函数
    fitness=np.array(op_table.iloc[:,0])
    chrom_best_temp = FindBest(chrom, fitness, N_chrom)  #寻找最优染色体
    if chrom_best_temp[-1] < chrom_best[j, -1]: #替换掉当前储存的最优
        chrom_best[j, :] = chrom_best_temp
        
    chrom, fitness =  EliteReplacement(chrom, chrom_best[j, :], fitness)
    fitness_best[j,0] = chrom_best[j, -1] #将当前最优存入矩阵当中
    
    
    # evolution strategy
    for iter in range(1, iter_all):
    #     mut, acr = adp_mut_acr(mut, acr, fitness,fitness_best[0,iter-1], fitness_ave[0,iter-1])  #自适应算子
        selected_index = TournamentSelection(chrom, fitness, 30) #tournament_size= individual*(5-20%)
        chrom = MutChrom(chrom[selected_index, :], mut, N, N_chrom, chrom_range, iter, iter_all) #变异
        chrom = AcrChrom(chrom[selected_index, :], acr, N, N_chrom) #交叉

        for i in range(0, N):
    #         P = chrom[i,0]
    #         d1 = chrom[i,1]
    #         d2 = chrom[i,2]
    #         H1 = chrom[i,3]
    #         wide = chrom[i,4]
    #         length = chrom[i,5]

    #         if ((2*d1+wide+2*H1) >= (P-1)) or ((H1-2*d2) <= 0):    #判断是否大于外正方形边长#            continue.....

            # curves predicted by NN (surrogate model)
            op_table.iloc[:, 1:] = pd.DataFrame(chrom).iloc[:, :]
            predicted_curves = model.predict(op_table.iloc[:, 1:])
            fitness = np.array(op_table.iloc[:, 0])

            # calculate error of the prediction
            target_error = [match_error(predicted_curves[k, :], inveser_data[j, :], target_scale)
                           for k in range(0, N)]
            op_table["error"] = target_error

            # sort based on error
            op_table = op_table.sort_values(by="error")

            if target_error_temp > np.min(op_table.iloc[:, 0]):
                target_error_temp = np.min(op_table.iloc[:, 0])
                best_pre_geo.iloc[0, :] = op_table.iloc[0, 1:]

        #display('iteration ' + str(iter))
#         display(op_table.head(5))
        chrom_best_temp = FindBest(chrom, fitness, N_chrom)  #寻找最优染色体
        if chrom_best_temp[-1] < chrom_best[j, -1]: #替换掉当前储存的最优
            chrom_best[j,:] = chrom_best_temp
        #替换掉最劣
        chrom, fitness =  EliteReplacement(chrom, chrom_best[j, :], fitness)
        fitness_best[j, iter] = chrom_best[j, -1] #将当前最优存入矩阵当中
        print(f"Iteration {iter}: Best fitness = {fitness_best[j, iter]}")
        if  np.min(fitness)<0.0001:
            break
            
    print(f"inverse data {j}: Best fitness = {np.min(fitness_best[j, :])}")


Iteration 1: Best fitness = 0.0026574591783649066
Iteration 2: Best fitness = 0.0026574591783649066
Iteration 3: Best fitness = 0.0026574591783649066
Iteration 4: Best fitness = 0.0026574591783649066
Iteration 5: Best fitness = 0.0026574591783649066
Iteration 6: Best fitness = 0.0026574591783649066
Iteration 7: Best fitness = 0.0026574591783649066
Iteration 8: Best fitness = 0.0026574591783649066
Iteration 9: Best fitness = 0.0026574591783649066
Iteration 10: Best fitness = 0.0026574591783649066
Iteration 11: Best fitness = 0.0026574591783649066
Iteration 12: Best fitness = 0.0026574591783649066
Iteration 13: Best fitness = 0.0026574591783649066
Iteration 14: Best fitness = 0.0026574591783649066
Iteration 15: Best fitness = 0.0026574591783649066
Iteration 16: Best fitness = 0.0026574591783649066
Iteration 17: Best fitness = 0.0026574591783649066
Iteration 18: Best fitness = 0.0026574591783649066
Iteration 19: Best fitness = 0.0026574591783649066
Iteration 20: Best fitness = 0.002352939

Iteration 65: Best fitness = 0.0021490962192669176
Iteration 66: Best fitness = 0.0021490962192669176
Iteration 67: Best fitness = 0.0021490962192669176
Iteration 68: Best fitness = 0.0021490962192669176
Iteration 69: Best fitness = 0.0021490962192669176
Iteration 70: Best fitness = 0.0021490962192669176
Iteration 71: Best fitness = 0.0021490962192669176
Iteration 72: Best fitness = 0.0021490962192669176
Iteration 73: Best fitness = 0.0021490962192669176
Iteration 74: Best fitness = 0.0021490962192669176
Iteration 75: Best fitness = 0.0021490962192669176
Iteration 76: Best fitness = 0.0021490962192669176
Iteration 77: Best fitness = 0.0021490962192669176
Iteration 78: Best fitness = 0.0021490962192669176
Iteration 79: Best fitness = 0.0021490962192669176
Iteration 80: Best fitness = 0.0021490962192669176
Iteration 81: Best fitness = 0.0021490962192669176
Iteration 82: Best fitness = 0.0021490962192669176
Iteration 83: Best fitness = 0.0021490962192669176
Iteration 84: Best fitness = 0.

Iteration 29: Best fitness = 0.002068745070921624
Iteration 30: Best fitness = 0.002068745070921624
Iteration 31: Best fitness = 0.002068745070921624
Iteration 32: Best fitness = 0.002068745070921624
Iteration 33: Best fitness = 0.002068745070921624
Iteration 34: Best fitness = 0.002060671794108848
Iteration 35: Best fitness = 0.002060671794108848
Iteration 36: Best fitness = 0.002060671794108848
Iteration 37: Best fitness = 0.002060671794108848
Iteration 38: Best fitness = 0.002060671794108848
Iteration 39: Best fitness = 0.002060671794108848
Iteration 40: Best fitness = 0.002060671794108848
Iteration 41: Best fitness = 0.002060671794108848
Iteration 42: Best fitness = 0.002060671794108848
Iteration 43: Best fitness = 0.002060671794108848
Iteration 44: Best fitness = 0.002060671794108848
Iteration 45: Best fitness = 0.002060671794108848
Iteration 46: Best fitness = 0.002060671794108848
Iteration 47: Best fitness = 0.002060671794108848
Iteration 48: Best fitness = 0.002060671794108848


Iteration 95: Best fitness = 0.00575928854347663
Iteration 96: Best fitness = 0.00575928854347663
Iteration 97: Best fitness = 0.00575928854347663
Iteration 98: Best fitness = 0.00575928854347663
Iteration 99: Best fitness = 0.00575928854347663
inverse data 4: Best fitness = 0.00575928854347663
Iteration 1: Best fitness = 0.0011409867929654846
Iteration 2: Best fitness = 0.0011409867929654846
Iteration 3: Best fitness = 0.0011409867929654846
Iteration 4: Best fitness = 0.0011409867929654846
Iteration 5: Best fitness = 0.0011409867929654846
Iteration 6: Best fitness = 0.0011409867929654846
Iteration 7: Best fitness = 0.0011409867929654846
Iteration 8: Best fitness = 0.0011409867929654846
Iteration 9: Best fitness = 0.0011409867929654846
Iteration 10: Best fitness = 0.0011409867929654846
Iteration 11: Best fitness = 0.0011409867929654846
Iteration 12: Best fitness = 0.0011409867929654846
Iteration 13: Best fitness = 0.0011409867929654846
Iteration 14: Best fitness = 0.0011409867929654846

Iteration 57: Best fitness = 0.0009599180749411758
Iteration 58: Best fitness = 0.0009599180749411758
Iteration 59: Best fitness = 0.0009599180749411758
Iteration 60: Best fitness = 0.0009599180749411758
Iteration 61: Best fitness = 0.0009599180749411758
Iteration 62: Best fitness = 0.0009599180749411758
Iteration 63: Best fitness = 0.0009599180749411758
Iteration 64: Best fitness = 0.0009599180749411758
Iteration 65: Best fitness = 0.0009599180749411758
Iteration 66: Best fitness = 0.0009599180749411758
Iteration 67: Best fitness = 0.0009599180749411758
Iteration 68: Best fitness = 0.0009599180749411758
Iteration 69: Best fitness = 0.0009599180749411758
Iteration 70: Best fitness = 0.0009599180749411758
Iteration 71: Best fitness = 0.0009599180749411758
Iteration 72: Best fitness = 0.0009599180749411758
Iteration 73: Best fitness = 0.0009599180749411758
Iteration 74: Best fitness = 0.0009599180749411758
Iteration 75: Best fitness = 0.0009599180749411758
Iteration 76: Best fitness = 0.

...

Iteration 43: Best fitness = 0.0009154074286933997
Iteration 44: Best fitness = 0.0009154074286933997
Iteration 45: Best fitness = 0.0009154074286933997
Iteration 46: Best fitness = 0.0009154074286933997
Iteration 47: Best fitness = 0.0009154074286933997
Iteration 48: Best fitness = 0.0009154074286933997
Iteration 49: Best fitness = 0.0009154074286933997
Iteration 50: Best fitness = 0.0009154074286933997
Iteration 51: Best fitness = 0.0009154074286933997
Iteration 52: Best fitness = 0.0009154074286933997
Iteration 53: Best fitness = 0.0009154074286933997
Iteration 54: Best fitness = 0.0009154074286933997
Iteration 55: Best fitness = 0.0009154074286933997
Iteration 56: Best fitness = 0.0009154074286933997
Iteration 57: Best fitness = 0.0009154074286933997
Iteration 58: Best fitness = 0.0009154074286933997
Iteration 59: Best fitness = 0.0009154074286933997
Iteration 60: Best fitness = 0.0009154074286933997
Iteration 61: Best fitness = 0.0009154074286933997
Iteration 62: Best fitness = 0.

Iteration 6: Best fitness = 0.0032547491473308445
Iteration 7: Best fitness = 0.0025348424461273875
Iteration 8: Best fitness = 0.0025348424461273875
Iteration 9: Best fitness = 0.0025348424461273875
Iteration 10: Best fitness = 0.0025348424461273875
Iteration 11: Best fitness = 0.0025348424461273875
Iteration 12: Best fitness = 0.0025348424461273875
Iteration 13: Best fitness = 0.0025348424461273875
Iteration 14: Best fitness = 0.0025348424461273875
Iteration 15: Best fitness = 0.0025348424461273875
Iteration 16: Best fitness = 0.0025348424461273875
Iteration 17: Best fitness = 0.0025348424461273875
Iteration 18: Best fitness = 0.0025348424461273875
Iteration 19: Best fitness = 0.0025348424461273875
Iteration 20: Best fitness = 0.0025348424461273875
Iteration 21: Best fitness = 0.0025348424461273875
Iteration 22: Best fitness = 0.0025348424461273875
Iteration 23: Best fitness = 0.0025348424461273875
Iteration 24: Best fitness = 0.0025348424461273875
Iteration 25: Best fitness = 0.0025

Iteration 67: Best fitness = 0.0020914613699347254
Iteration 68: Best fitness = 0.0020914613699347254
Iteration 69: Best fitness = 0.0020914613699347254
Iteration 70: Best fitness = 0.0020914613699347254
Iteration 71: Best fitness = 0.0020914613699347254
Iteration 72: Best fitness = 0.0020914613699347254
Iteration 73: Best fitness = 0.0020914613699347254
Iteration 74: Best fitness = 0.0020914613699347254
Iteration 75: Best fitness = 0.0020914613699347254
Iteration 76: Best fitness = 0.0020914613699347254
Iteration 77: Best fitness = 0.0020914613699347254
Iteration 78: Best fitness = 0.0020914613699347254
Iteration 79: Best fitness = 0.0020914613699347254
Iteration 80: Best fitness = 0.0020914613699347254
Iteration 81: Best fitness = 0.0020914613699347254
Iteration 82: Best fitness = 0.0020914613699347254
Iteration 83: Best fitness = 0.0020914613699347254
Iteration 84: Best fitness = 0.0020914613699347254
Iteration 85: Best fitness = 0.0020914613699347254
Iteration 86: Best fitness = 0.

Iteration 30: Best fitness = 0.0006801456555323422
Iteration 31: Best fitness = 0.0006801456555323422
Iteration 32: Best fitness = 0.0006801456555323422
Iteration 33: Best fitness = 0.0006801456555323422
Iteration 34: Best fitness = 0.0006801456555323422
Iteration 35: Best fitness = 0.0006801456555323422
Iteration 36: Best fitness = 0.0006801456555323422
Iteration 37: Best fitness = 0.0006801456555323422
Iteration 38: Best fitness = 0.0006801456555323422
Iteration 39: Best fitness = 0.0006801456555323422
Iteration 40: Best fitness = 0.0006801456555323422
Iteration 41: Best fitness = 0.0006801456555323422
Iteration 42: Best fitness = 0.0006801456555323422
Iteration 43: Best fitness = 0.0006801456555323422
Iteration 44: Best fitness = 0.0006801456555323422
Iteration 45: Best fitness = 0.0006801456555323422
Iteration 46: Best fitness = 0.0006801456555323422
Iteration 47: Best fitness = 0.0006801456555323422
Iteration 48: Best fitness = 0.0006801456555323422
Iteration 49: Best fitness = 0.

Iteration 91: Best fitness = 0.0005355644843993938
Iteration 92: Best fitness = 0.0005355644843993938
Iteration 93: Best fitness = 0.0005355644843993938
Iteration 94: Best fitness = 0.0005355644843993938
Iteration 95: Best fitness = 0.0005355644843993938
Iteration 96: Best fitness = 0.0005355644843993938
Iteration 97: Best fitness = 0.0005355644843993938
Iteration 98: Best fitness = 0.0005355644843993938
Iteration 99: Best fitness = 0.0005355644843993938
inverse data 30: Best fitness = 0.0005355644843993938
Iteration 1: Best fitness = 0.0012766716344013317
Iteration 2: Best fitness = 0.0012766716344013317
Iteration 3: Best fitness = 0.0012766716344013317
Iteration 4: Best fitness = 0.0012766716344013317
Iteration 5: Best fitness = 0.0012766716344013317
Iteration 6: Best fitness = 0.0012766716344013317
Iteration 7: Best fitness = 0.0012766716344013317
Iteration 8: Best fitness = 0.0012766716344013317
Iteration 9: Best fitness = 0.0012766716344013317
Iteration 10: Best fitness = 0.001276

In [11]:
np.savetxt('./loss/'+ 'GA-error_2'+'.txt', np.array(fitness_best), delimiter=',')